## Working the CSVs and Excel sheets

### Train dataset

In [1]:
# Exploring the 'training' dataset
import pandas as pd
df = pd.read_csv("datasets/txt2sql_alerce_train_v3_1.csv")
df.head()

,req_id,request,table_info,external_knowledge,domain_knowledge,gold_query,difficulty,type,nested_type,rephrased_request,rephrased_request_gpt-3.5-turbo-0125_t0.4,rephrased_request_gpt-4o-2024-05-13_t0.2
0,13,Give me all the SNe that were first detected b...,"['object', 'probability']",\r\n-- mjd date for December = 59914.0\r\n-- m...,\r\n-- Super Nova (SNe) is a large explosion t...,"\r\nSELECT\r\n object.oid, probability.clas...",simple,object,none,NaN,NaN,NaN
1,10,Get the object identifiers and probabilities i...,"['probability', 'object']",0,0,"\r\nSELECT\r\n sq1.oid, sq1.probability as SN...",medium,object,simple,NaN,NaN,NaN
2,15,"Get the object identifiers, probabilities in t...","['object', 'probability', 'detection', 'magstat']",\r\n-- mjd date for September 01 = 60188.0\r\n...,\r\n-- A fast riser is defined as an object wh...,"\r\nSELECT\r\n sq.oid, sq.probability, sq.c...",advanced,other,multi,NaN,NaN,NaN
3,4,"Get the object identifier, candidate identifie...","['object', 'probability', 'magstat', 'detection']",\r\n-- mjd date for the start of the year 2019...,0,"\r\nSELECT\r\n sq.oid, sq.fid, sq.dmdt_first,...",advanced,other,multi,NaN,NaN,NaN
4,25,Query objects within 10 degress of the next po...,"['probability', 'object']",\r\n-- mjd date for February 01 = 59976.0\r\n-...,0,"\r\nWITH catalog ( source_id, ra, dec) AS (\r\...",advanced,spatial,simple,NaN,NaN,NaN


In [2]:
df[df["req_id"] == 15]["request"].item()

"Get the object identifiers, probabilities in the stamp classifier and light curves (only detections) for objects whose highest probability in the stamp classifier is obtained for class SN, that had their first detection in the first 2 days of september, and that qualify as fast risers. Also, return the filter ID and candid of the detections, and filter ID of magstat as 'magstat_fid', with the initial rise estimate. Order the results by the oid."

In [3]:
print("External knowledge for the train dataset")
vc_df = df["external_knowledge"].value_counts().reset_index()
vc_df.columns = ["Value", "Count"]

vc_df

External knowledge for the train dataset


,Value,Count
0,0,29
1,\r\n-- mjd date for February 01 = 59976.0\r\n-...,2
2,\r\n-- mjd date for December = 59914.0\r\n-- m...,1
3,\r\n-- ndet represents the number of detection...,1
4,Note that 'fainter' means 'larger' for magnitudes,1
5,Note that the spatial query receives coordinat...,1
6,"Since no classifier version is specified, ZTF ...",1
7,Note the join between tables must be in oid an...,1
8,"Since no feature version is specified, the com...",1
9,\r\n-- sgmag1 represents the g-band PSF magnit...,1


In [4]:
print("Domain knowledge for the train dataset")
vc_df_domain = df["domain_knowledge"].value_counts().reset_index()
vc_df_domain.columns = ["Value", "Count"]

vc_df_domain

Domain knowledge for the train dataset


,Value,Count
0,0,19
1,\r\n-- Super Nova (SNe) is a large explosion t...,1
2,\r\n-- A fast riser is defined as an object wh...,1
3,\r\n-- ss_ztf: closest MPC object as reported ...,1
4,\r\n-- sgscore1: Morphological star/galaxy sco...,1
5,\r\n-- delta_mag_fid: Difference between maxim...,1
6,\r\n-- Multiband_period: Period obtained using...,1
7,\r\n-- The galactic coordinate system locates ...,1
8,\r\n-- ps1 catalog is a catalog from the Panor...,1


### Test dataset

In [5]:
df_test = pd.read_csv("datasets/txt2sql_alerce_test_v3_1.csv")
df_test.head()

,req_id,request,table_info,external_knowledge,domain_knowledge,gold_query,difficulty,type,nested_type,rephrased_request,rephrased_request_gpt-3.5-turbo-0125_t0.4,rephrased_request_gpt-4o-2024-05-13_t0.2
0,8,Query all objects that were first classified a...,"['probability', 'object']",\r\n-- mjd date for august 17 = 60173.0\r\n-- ...,NaN,\r\nSELECT\r\n *\r\nFROM\r\n object\r\nI...,simple,object,none,NaN,NaN,NaN
1,27,"Return the oids, meanra, meandec, ndet, firstm...","['probability', 'object']",\r\n-- object.ndet represents the total number...,NaN,"\r\nSELECT\r\n object.oid, object.meanra, o...",simple,object,simple,NaN,NaN,NaN
2,17,Return the objects classified as AGN that show...,"['probability', 'object', 'detection']",\r\n-- firstmjd represents the time of the fir...,NaN,\r\nSELECT\r\n *\r\nFROM\r\n (\r\nSELECT *\r...,advanced,object,tree,NaN,NaN,NaN
3,37,"""Query the top 200 objects classified SNIa acc...",['probability'],NaN,NaN,\r\nSELECT\r\n *\r\nFROM\r\n probability\r...,simple,object,none,NaN,NaN,NaN
4,34,Given this list of oids ['ZTF17aaadpsi' 'ZTF19...,['feature'],\r\n-- feature.name can be 'Amplitude'\r\n-- f...,\r\n-- Multiband_period: Period obtained using...,\r\nSELECT\r\n *\r\nFROM\r\n feature\r\n...,simple,other,none,NaN,NaN,NaN


In [6]:
print("External knowledge for the test dataset")
vc_df_test = df_test["external_knowledge"].value_counts().reset_index()
vc_df_test.columns = ["Value", "Count"]

vc_df_test

External knowledge for the test dataset


,Value,Count
0,0,4
1,Note the join between tables must be in oid an...,2
2,Note that 'fainter' means 'larger' for magnitudes,2
3,"Since no classifier version is specified, ZTF ...",2
4,\r\n-- object.ndet represents the total number...,1
5,Since no classifier version is specified for '...,1
6,I had to use 'cast' for magnitudes in the 'for...,1
7,Note that the corrected magnitude has both e_m...,1
8,Requires using the 'xmatch' table to obtain th...,1
9,Requires using the 'xmatch' table to obtain th...,1


In [7]:
print("Domain knowledge for the train dataset")
vc_df_domain_test = df_test["domain_knowledge"].value_counts().reset_index()
vc_df_domain_test.columns = ["Value", "Count"]

vc_df_domain_test

Domain knowledge for the train dataset


,Value,Count
0,\r\n-- Multiband_period: Period obtained using...,1
1,\r\n-- Amplitude: Half of the difference betwe...,1
2,\r\n-- A fast riser is defined as an object wh...,1
3,"\r\n-- Asteroids move, so it is necessary to c...",1
4,\r\n-- SNe refers to all types of SuperNova\r\n,1


### Additional info

In [8]:
# Reading the excel file
excel = pd.read_excel("datasets/SQLusecases_alerce.xlsx", 
                      sheet_name="examples_alerce_usecasesV3_1")

# Cleaning the excel a bit
excel = excel.drop(columns=["Unnamed: 0.1", "Unnamed: 0"])
excel.head()

,req_id,request,table_info,external_knowledge,domain_knowledge,gold_query,difficulty,type,nested_type,Set,python_format
0,0,Get objects that are likely to be YSOs (possib...,"['probability', 'feature']",\n-- feature.name can be 'Multiband_period'\n-...,\n-- Multiband_period: Period obtained using t...,"\nSELECT\n oid, probability, value, name, fid...",advanced,object,tree,Train,"sub_query_1='''\nSELECT\n feature.oid, prob_o..."
1,1,Get all the objects classified as AGN with a p...,"['object', 'probability', 'feature', 'magstat']",\n-- object.ndet represents the number of dete...,\n-- Amplitude: Half of the difference between...,"\nSELECT\n sq.oid, sq.value, sq.name, sq.fid ...",advanced,object,tree,Test,\nsub_query_object='''\nSELECT\n object.oid...
2,2,Give me the objects classified as YSO by their...,['probability'],NaN,NaN,"\nSELECT\n oid, probability\nFROM\n prob...",simple,object,none,Train,"query='''\nSELECT\n oid, probability\nFROM\..."
3,3,Give me the objects classified as YSO by the l...,"['object', 'probability']","\n-- last june in mjd date: [start=60096.0, en...",NaN,\nSELECT\n *\nFROM\n probability\nWHERE\...,simple,object,simple,Train,query=f'''\nSELECT\n *\nFROM\n probabili...
4,4,"Get the object identifier, candidate identifie...","['object', 'probability', 'magstat', 'detection']",\n-- mjd date for the start of the year 2019 =...,NaN,"\nSELECT\n sq.oid, sq.fid, sq.dmdt_first,\n ...",advanced,other,multi,Train,# objects classified as SN II with probability...


## Tests

### Gold values and query to test

In [9]:
# Select a query
query = "Give me all the SNe that were first detected between december first 2022 and september first 2023. Return the probability class, the last and the first detection date and the oids of the objects."

# Check if the query is in the excel file
if query in excel["request"].to_list():
    # Obtain the gold SQL query and Python query
    sql_gold = excel[excel["request"] == query]["gold_query"].item()
    python_gold = excel[excel["request"] == query]["python_format"].item()

    # Obtain the necessary tables
    gold_tables = excel[excel["request"] == query]["table_info"].item()

    # Print all in orderly fashion
    print("Gold values\n")
    print("Tables needed for the query:")
    print(gold_tables + "\n")
    print("SQL gold query:")
    print(sql_gold)
    print("Python gold query:\n")
    print(python_gold)
    
else:
    # Obtain the gold SQL query
    sql_gold = df[df["request"] == query]["gold_query"].item()

    # Obtain the necessary tables
    gold_tables = df[df["request"] == query]["table_info"].item()

    # Print all in orderly fashion
    print("Gold values\n")
    print("Tables needed for the query:")
    print(gold_tables + "\n")
    print("SQL gold query:")
    print(sql_gold)

Gold values

Tables needed for the query:
['object', 'probability']

SQL gold query:

SELECT
    object.oid, probability.class_name, object.lastmjd, object.firstmjd
FROM
    object INNER JOIN
    probability
    ON object.oid = probability.oid
WHERE
    probability.classifier_name='lc_classifier'
    AND probability.class_name IN ('SNIa', 'SNIbc', 'SNII', 'SLSN')
    AND probability.ranking = 1
    AND object.firstmjd < 60217.0
    AND object.firstmjd > 59914.0



In [10]:
# Running the gold query
from secret.config import SQL_URL
import requests
import sqlalchemy as sa

# Setup params for query engine
params = requests.get(SQL_URL).json()['params']
engine = sa.create_engine(f"postgresql+psycopg2://{params['user']}:{params['password']}@{params['host']}/{params['dbname']}")
engine.begin()

resultGold = pd.read_sql_query(sql_gold, con=engine)
resultGold

,oid,class_name,lastmjd,firstmjd
0,ZTF23aaocetr,SNIa,60133.210428,60109.208229
1,ZTF23aamzkrs,SNIa,60124.189074,60097.242882
2,ZTF23aamxbpd,SNIa,60120.218900,60100.190417
3,ZTF23aafwzmi,SNIa,60078.406505,60050.470231
4,ZTF23aackdba,SNII,60073.174653,59992.212407
...,...,...,...,...
4542,ZTF23aagaoax,SNII,60761.252708,60049.210880
4543,ZTF23aaefijg,SNII,60786.461586,60036.459398
4544,ZTF20aanlftl,SNII,60786.408137,59977.563657
4545,ZTF23aaavavg,SNIa,60770.256586,59967.353044


### Trying out the pipelines

#### Singular

In [10]:
from pipeline.queryPipeline import queryPipeline

# Working query
query = "Get the object identifiers, probabilities in the stamp classifier and light curves (only detections) for objects whose highest probability in the stamp classifier is obtained for class SN, that had their first detection in the first 2 days of september, and that qualify as fast risers. Also, return the filter ID and candid of the detections, and filter ID of magstat as 'magstat_fid', with the initial rise estimate. Order the results by the oid."

# Model to use
#model = "claude-3-5-sonnet-20240620"
model = "gpt-4o"

# Format for the pipeline
lang_type = "sql"
max_tokens = 10000

In [ ]:
from pipeline.process import *
from prompts.base.prompts import *

# Prompt dictionary guideline and used by Jorge
prompts = {
    "Schema Linking": {
        "base_prompt": tables_linking_prompt_V2,
        "context1": schema_all_cntxV1,
        "context2": schema_all_cntxV2_indx,
        "context3": schema_all_cntxV2,
    },
    "Classify": {
        "base_prompt": diff_class_prompt_v7,
        "final_instructions": final_instructions_diff_v2
    },
    "Decomposition": {
        "simple": {
            "query_task": simple_query_task_v2,
            "query_context": simple_query_cntx,
            "external_knowledge": "placeholder",
            "domain_knowledge": "placeholder",
            "query_instructions": simple_query_instructions_v2
        },
        "medium": {
            "decomp_plan": {
                "base_prompt": medium_decomp_prompt,
                "decomp_task": medium_decomp_task_v3 + gpt4turbo1106_decomposed_prompt_2,
                "query_context": medium_query_cntx,
                "query_instructions": medium_query_instructions_1_v2
            },
            "decomp_gen": {
                "sql": {
                    "base_prompt": medium_decomp_gen,
                    "query_task": medium_query_task_v2,
                    "query_instructions": medium_query_instructions_2_v2,
                },
                "python": {
                    "base_prompt": "placeholder",
                    "query_task": "placeholder",
                    "query_instructions": "placeholder",
                }
            }
        },
        "advanced": {
            "decomp_plan": {
                "base_prompt": adv_decomp_prompt,
                "decomp_task": adv_decomp_task_v3 + gpt4turbo1106_decomposed_prompt_2,
                "query_context": adv_query_cntx,
                "query_instructions": adv_query_instructions_1_v3
            },
            "decomp_gen": {
                "sql": {
                    "base_prompt": adv_decomp_gen,
                    "query_task": adv_query_task_v2,
                    "query_instructions": adv_query_instructions_2_v3,
                },
                "python": {
                    "base_prompt": "placeholder",
                    "query_task": "placeholder",
                    "query_instructions": "placeholder",
                }
            }
        }
    },
    "Direct": {
        "base_prompt": {
            "general_task": general_taskv18,
            "general_context": general_contextv15,
            "final_instructions": final_instructions_v19
        },
        "request_prompt": {
            "external_knowledge": "placeholder",
            "domain_knowledge": "placeholder"
        }
    }
}

# Updating ext_kn and dom_kn
prompts["Decomposition"]["simple"]["external_knowledge"] = df[df["request"] == query]["external_knowledge"].item()
prompts["Decomposition"]["simple"]["domain_knowledge"] = df[df["request"] == query]["domain_knowledge"].item()
prompts["Direct"]["request_prompt"]["external_knowledge"] = df[df["request"] == query]["external_knowledge"].item()
prompts["Direct"]["request_prompt"]["domain_knowledge"] = df[df["request"] == query]["domain_knowledge"].item()

In [ ]:
# Pipeline object
pipe = queryPipeline(query, model, lang_type, max_tokens, prompts)

# Schema Linking
schema_usage = pipe.schema_linking()

# Classification
class_usage = pipe.classify()

# Decomposition
pipe.decomposition()

# Direct prompt
pipe.direct()

# Generating the queries
decomp_gen_query = pipe.query_generation()
print("Query generated with decomposition prompts")
print(decomp_gen_query)
print("-"*50)
direct_gen_query = pipe.query_generation()
print("Query generated with direct prompts")
print(direct_gen_query)

Query generated with decomposition prompts

-- Subquery to get object identifiers (oids) that qualify as fast risers
WITH fast_risers AS (
    SELECT DISTINCT oid
    FROM magstat
    WHERE dmdt_first <= -0.25
),

-- Subquery to get object identifiers (oids) with first detection in the first 2 days of September
first_september_detections AS (
    SELECT oid
    FROM object
    WHERE firstmjd BETWEEN 60188.0 AND 60189.0
),

-- Subquery to get object identifiers (oids) with highest probability in the stamp classifier for class SN
high_prob_sn AS (
    SELECT oid
    FROM probability
    WHERE classifier_name = 'stamp_classifier'
    AND class_name = 'SN'
    AND ranking = 1
)

-- Main query to get the required information
SELECT 
    d.oid,
    p.probability,
    d.fid,
    d.candid,
    m.fid AS magstat_fid,
    m.dmdt_first
FROM detection d
JOIN fast_risers fr ON d.oid = fr.oid
JOIN first_september_detections fsd ON d.oid = fsd.oid
JOIN high_prob_sn hps ON d.oid = hps.oid
JOIN probabil

#### Multiple

##### Query generation

In [ ]:
from pipeline.queryPipeline import queryPipeline
from pipeline.process import *
from prompts.base.prompts import *

# Prompt dictionary guideline and used by Jorge
prompts = {
    "Schema Linking": {
        "base_prompt": tables_linking_prompt_V2,
        "context1": schema_all_cntxV1,
        "context2": schema_all_cntxV2_indx,
        "context3": schema_all_cntxV2,
    },
    "Classify": {
        "base_prompt": diff_class_prompt_v7,
        "final_instructions": final_instructions_diff_v2
    },
    "Decomposition": {
        "simple": {
            "query_task": simple_query_task_v2,
            "query_context": simple_query_cntx,
            "external_knowledge": "placeholder",
            "domain_knowledge": "placeholder",
            "query_instructions": simple_query_instructions_v2
        },
        "medium": {
            "decomp_plan": {
                "base_prompt": medium_decomp_prompt,
                "decomp_task": medium_decomp_task_v3 + gpt4turbo1106_decomposed_prompt_2,
                "query_context": medium_query_cntx,
                "query_instructions": medium_query_instructions_1_v2
            },
            "decomp_gen": {
                "sql": {
                    "base_prompt": medium_decomp_gen,
                    "query_task": medium_query_task_v2,
                    "query_instructions": medium_query_instructions_2_v2,
                },
                "python": {
                    "base_prompt": "placeholder",
                    "query_task": "placeholder",
                    "query_instructions": "placeholder",
                }
            }
        },
        "advanced": {
            "decomp_plan": {
                "base_prompt": adv_decomp_prompt,
                "decomp_task": adv_decomp_task_v3 + gpt4turbo1106_decomposed_prompt_2,
                "query_context": adv_query_cntx,
                "query_instructions": adv_query_instructions_1_v3
            },
            "decomp_gen": {
                "sql": {
                    "base_prompt": adv_decomp_gen,
                    "query_task": adv_query_task_v2,
                    "query_instructions": adv_query_instructions_2_v3,
                },
                "python": {
                    "base_prompt": "placeholder",
                    "query_task": "placeholder",
                    "query_instructions": "placeholder",
                }
            }
        }
    },
    "Direct": {
        "base_prompt": {
            "general_task": general_taskv18,
            "general_context": general_contextv15,
            "final_instructions": final_instructions_v19
        },
        "request_prompt": {
            "external_knowledge": "placeholder",
            "domain_knowledge": "placeholder"
        }
    }
}

# Updating ext_kn and dom_kn
prompts["Decomposition"]["simple"]["external_knowledge"] = df[df["request"] == query]["external_knowledge"].item()
prompts["Decomposition"]["simple"]["domain_knowledge"] = df[df["request"] == query]["domain_knowledge"].item()
prompts["Direct"]["request_prompt"]["external_knowledge"] = df[df["request"] == query]["external_knowledge"].item()
prompts["Direct"]["request_prompt"]["domain_knowledge"] = df[df["request"] == query]["domain_knowledge"].item()

In [ ]:
# Model to use
#model = "claude-3-5-sonnet-20240620"
model = "gpt-4o"

# Format for the pipeline
lang_type = "sql"
max_tokens = 10000

# Path
prompts_path = "final_prompts/prompts_v1.json"

# Pipeline object
pipe = queryPipeline(query, model, lang_type, max_tokens, prompts_path)

# Trying out the new query pipeline with only 1 query
df_1 = df[df["request"] == query]
# Trying out the new query pipeline with 3 queries
df_2 = df.loc[:2]
pipe.run_experiments(df_2, total_exps=2, restart=True)

Restarting
Query ID: 10, Run ID: 1
Running pipeline
Saving backup
Query ID: 10, Run ID: 2
Running pipeline
Saving backup
Query ID: 15, Run ID: 1
Running pipeline
Saving backup
Query ID: 15, Run ID: 2
Running pipeline
Saving backup
Saving all


In [ ]:
# Making all the predictions at once for test
pipe.run_experiments(df=df_test, total_exps=10)

Running process
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving backup
Running pipeline
Saving

In [ ]:
# Applying the restart
pipe.run_experiments(df=df_test, total_exps=10, restart=True)

Restarting
Query ID: 39, Run ID: 2
Running pipeline
Saving backup
Query ID: 39, Run ID: 3
Running pipeline
Saving backup
Query ID: 39, Run ID: 4
Running pipeline
Saving backup
Query ID: 39, Run ID: 5
Running pipeline
Saving backup
Query ID: 39, Run ID: 6
Running pipeline
Saving backup
Query ID: 39, Run ID: 7
Running pipeline
Saving backup
Query ID: 39, Run ID: 8
Running pipeline
Saving backup
Query ID: 39, Run ID: 9
Running pipeline
Saving backup
Query ID: 39, Run ID: 10
Running pipeline
Saving backup
Saving all


##### Metrics generation

In [ ]:
from pipeline.metricsPipeline import metricsPipeline

# Model to use
#model = "claude-3-5-sonnet-20240620"
model = "gpt-4o"

# Format for the pipeline
lang_type = "sql"
max_tokens = 10000

# Paths
pred_path = "experiments/preds_gpt-4o_2025-04-29T14-55-41.csv"
prompts_path = "final_prompts/prompts_v1.json"

# Trying out the metrics pipeline with only 1 query
df_1 = df[df["request"] == query]
# Trying out the metrics pipeline with 3 queries
df_2 = df.loc[:2]
metrics = metricsPipeline(model, lang_type, max_tokens, df_2, 
                          t_conn=2, n_tries=3, direct=False, self_corr=False, 
                          self_corr_prompts={}, prompts_path=prompts_path)
metrics.run_metrics(sql_preds_path=pred_path, total_exps=1, restart=True)

Tiempos direct / no self-correction (18/05/2025)

279m 29.9s
12m 46.5s

Tiempos direct / self-correction (19/05/2025)

5m 4.7s
409m 22s
139m 15.1s

In [10]:
from pipeline.metricsPipeline import metricsPipeline

# Model to use
#model = "claude-3-5-sonnet-20240620"
model = "gpt-4o"

# Format for the pipeline
lang_type = "sql"
max_tokens = 10000

# Paths
pred_path = "experiments/preds_gpt-4o_2025-04-29T14-55-41.csv"
prompts_path = "final_prompts/prompts_v1.json"

# Running all the metrics of the test set
metrics = metricsPipeline(model, lang_type, max_tokens, 
                          t_conn=2, n_tries=3, direct=True, self_corr=True, 
                          self_corr_prompts={}, prompts_path=prompts_path)
metrics.run_metrics(sql_preds_path=pred_path, df=df_test, total_exps=10, restart=True)

2025-05-19 17:10:44,268 - INFO - Restarting
2025-05-19 17:10:44,497 - INFO - Query ID: 102, Run ID: 5
2025-05-19 17:10:44,500 - INFO - Getting the gold values to compare
2025-05-19 17:10:48,173 - INFO - Query ID: 102, Run ID: 5, Query gold:               oid  fid  stellar  corrected  ndet  ndubious  dmdt_first  \
0    ZTF18acnnxyp    1    False       True   220         0         NaN   
1    ZTF18acnnxyp    2    False       True   187         0         NaN   
2    ZTF19aaslszp    1    False       True    14         0         NaN   
3    ZTF19aaslszp    2    False       True    38         0         NaN   
4    ZTF19accprws    1    False       True   100         0         NaN   
..            ...  ...      ...        ...   ...       ...         ...   
103  ZTF18abtmeds    2    False       True   236         1         NaN   
104  ZTF18acbycof    1     True       True     6         0         NaN   
105  ZTF18acbycof    2     True       True    10         0         NaN   
106  ZTF19acbuwdb  

Running SQL exception in run_query: (psycopg2.errors.UndefinedColumn) column allwise.oid does not exist
LINE 5: INNER JOIN allwise ON magstat.oid = allwise.oid
                                            ^

[SQL: 
SELECT magstat.*
FROM magstat
INNER JOIN probability ON magstat.oid = probability.oid
INNER JOIN allwise ON magstat.oid = allwise.oid
WHERE probability.class_name = 'AGN'
  AND probability.classifier_name = 'lc_classifier'
  AND probability.probability > 0.6
  AND probability.ranking = 1
  AND allwise.w1mpro BETWEEN 12 AND 13
LIMIT 50
]
(Background on this error at: https://sqlalche.me/e/20/f405)


2025-05-19 17:12:09,474 - INFO - Saving backup
2025-05-19 17:12:09,838 - INFO - Query ID: 81, Run ID: 0
2025-05-19 17:12:13,349 - INFO - Query ID: 81, Run ID: 0, Query gold:             oid      dist
0  ZTF21abbrgtf  0.500686
2025-05-19 17:12:13,372 - INFO - Saving backup
2025-05-19 17:12:13,722 - INFO - Saving backup
2025-05-19 17:12:14,026 - INFO - Query ID: 81, Run ID: 1
2025-05-19 17:12:14,027 - INFO - Getting the gold values to compare
2025-05-19 17:12:17,667 - INFO - Query ID: 81, Run ID: 1, Query gold:             oid      dist
0  ZTF21abbrgtf  0.500686
2025-05-19 17:12:17,676 - INFO - Getting the request for self correction
2025-05-19 17:12:17,683 - INFO - Self-correction
2025-05-19 17:13:29,641 - INFO - Saving backup
2025-05-19 17:13:30,002 - INFO - Query ID: 81, Run ID: 2
2025-05-19 17:13:30,005 - INFO - Getting the gold values to compare
2025-05-19 17:13:33,602 - INFO - Query ID: 81, Run ID: 2, Query gold:             oid      dist
0  ZTF21abbrgtf  0.500686
2025-05-19 17:13:

Running SQL exception in run_query: (psycopg2.errors.QueryCanceled) canceling statement due to statement timeout

[SQL: SELECT
    *
FROM
    magstat
WHERE
    ndet >= 10
    AND magmedian > 19
    AND maglast < 17
ORDER BY oid
LIMIT 30
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


2025-05-19 17:20:54,591 - INFO - Query ID: 61, Run ID: 0, Query gold:              oid  fid  stellar  corrected  ndet  ndubious  dmdt_first  \
0   ZTF17aaaazob    1    False       True   595         0         NaN   
1   ZTF17aaadaqj    1    False       True   109         2         NaN   
2   ZTF17aaadzhc    1    False       True   511         1         NaN   
3   ZTF17aaagzbe    1    False       True    44         0         NaN   
4   ZTF17aaahyqe    1    False       True   142         0         NaN   
5   ZTF17aaaihhg    1    False       True    12         0         NaN   
6   ZTF17aaakimh    2    False       True    31         0         NaN   
7   ZTF17aaakkku    1    False       True    12         0         NaN   
8   ZTF17aaaskut    1    False       True    32         0   -0.324131   
9   ZTF17aaastit    1    False       True   113         0         NaN   
10  ZTF17aaaswvx    1    False       True   626         2         NaN   
11  ZTF17aaatnbo    1    False       True    48       

...

2025-05-19 19:21:00,922 - INFO - Saving backup
2025-05-19 19:21:01,738 - INFO - Query ID: 39, Run ID: 8
2025-05-19 19:21:01,739 - INFO - Getting the gold values to compare
2025-05-19 19:21:05,479 - INFO - Query ID: 39, Run ID: 8, Query gold:               oid      meanra    meandec               candid  f_id  \
0    ZTF17aaaaanb  324.816860  45.593583   537475640115015008     2   
1    ZTF17aaaaanb  324.816860  45.593583   607270780115015027     2   
2    ZTF17aaaaanb  324.816860  45.593583   613325340115010003     2   
3    ZTF17aaaaanb  324.816860  45.593583   536383390115015014     2   
4    ZTF17aaaaanb  324.816860  45.593583   529390300115015005     2   
..            ...         ...        ...                  ...   ...   
995  ZTF17aaadgdr  116.399044  32.744831  1496215692515015015     2   
996  ZTF17aaadgdr  116.399044  32.744831  1563162532515010015     2   
997  ZTF17aaadgdr  116.399044  32.744831  1582202532515015009     2   
998  ZTF17aaadgdr  116.399044  32.744831  156316

Running SQL exception in run_query: (psycopg2.errors.QueryCanceled) canceling statement due to statement timeout

[SQL: 
WITH probability_subquery AS (
    SELECT oid
    FROM probability
    WHERE class_name = 'CV/Nova'
      AND ranking = 1
      AND classifier_name = 'lc_classifier_stochastic'
),
object_subquery AS (
    SELECT oid
    FROM object
    WHERE ndet <= 50
),
detection_subquery AS (
    SELECT d.oid, d.candid, d.fid AS f_id, d.ra, d.dec, d.magpsf, d.sigmapsf, d.magpsf_corr
    FROM detection d
    INNER JOIN object_subquery o ON d.oid = o.oid
    WHERE d.fid = 2
),
magstat_subquery AS (
    SELECT m.oid, m.magmax_corr, m.magmin_corr
    FROM magstat m
    INNER JOIN object_subquery o ON m.oid = o.oid
    WHERE m.fid = 2
)
SELECT d.oid, d.candid, d.f_id, d.ra, d.dec, d.magpsf, d.sigmapsf, d.magpsf_corr, 
       (m.magmax_corr - m.magmin_corr) AS max_rise
FROM detection_subquery d
INNER JOIN magstat_subquery m ON d.oid = m.oid
INNER JOIN probability_subquery p ON d.oid = p

2025-05-19 19:25:40,738 - INFO - Saving backup
2025-05-19 19:25:41,558 - INFO - Query ID: 39, Run ID: 10
2025-05-19 19:25:41,559 - INFO - Getting the gold values to compare
2025-05-19 19:25:45,302 - INFO - Query ID: 39, Run ID: 10, Query gold:               oid      meanra    meandec               candid  f_id  \
0    ZTF17aaaaanb  324.816860  45.593583   537475640115015008     2   
1    ZTF17aaaaanb  324.816860  45.593583   607270780115015027     2   
2    ZTF17aaaaanb  324.816860  45.593583   613325340115010003     2   
3    ZTF17aaaaanb  324.816860  45.593583   536383390115015014     2   
4    ZTF17aaaaanb  324.816860  45.593583   529390300115015005     2   
..            ...         ...        ...                  ...   ...   
995  ZTF17aaadgdr  116.399044  32.744831  1496215692515015015     2   
996  ZTF17aaadgdr  116.399044  32.744831  1563162532515010015     2   
997  ZTF17aaadgdr  116.399044  32.744831  1582202532515015009     2   
998  ZTF17aaadgdr  116.399044  32.744831  1563

Running SQL exception in run_query: (psycopg2.errors.QueryCanceled) canceling statement due to statement timeout

[SQL: 
WITH probability_subquery AS (
    SELECT oid
    FROM probability
    WHERE class_name = 'CV/Nova'
      AND classifier_name = 'lc_classifier_stochastic'
      AND ranking = 1
),
object_subquery AS (
    SELECT oid
    FROM object
    WHERE ndet <= 50
),
detection_subquery AS (
    SELECT d.oid, d.candid, d.ra, d.dec, d.magpsf_corr, d.magpsf
    FROM detection d
    INNER JOIN object_subquery o ON d.oid = o.oid
    WHERE d.fid = 2
),
magstat_subquery AS (
    SELECT m.oid, m.magmax_corr, m.magmin_corr
    FROM magstat m
    INNER JOIN object_subquery o ON m.oid = o.oid
    WHERE m.fid = 2
)
SELECT d.oid, d.candid, d.ra, d.dec, d.magpsf_corr, d.magpsf, 
       (m.magmax_corr - m.magmin_corr) AS max_rise
FROM detection_subquery d
INNER JOIN magstat_subquery m ON d.oid = m.oid
INNER JOIN probability_subquery p ON d.oid = p.oid
ORDER BY d.oid
LIMIT 1000
]
(Background on

2025-05-19 19:29:57,290 - INFO - Saving backup
2025-05-19 19:29:58,107 - INFO - Process ended. Saving it all


In [15]:
metrics.new_df

,code_tag,llm_used,prompt_version,query_id,query_run,sql_query,tab_schema,label,query_gen_time,query_gen_date,query_results,query_error,sql_time,sql_date,r_row,p_row,r_col,p_col,N_perfect_row,N_perfect_col
0,0.16.1,None,None,40,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,0.16.1,gpt-4o,v1,8,1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,0.16.1,gpt-4o,v1,8,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,0.16.1,gpt-4o,v1,8,3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,0.16.1,gpt-4o,v1,8,4,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,0.16.1,gpt-4o,v1,39,6,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
568,0.16.1,gpt-4o,v1,39,7,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
569,0.16.1,gpt-4o,v1,39,8,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
570,0.16.1,gpt-4o,v1,39,9,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [11]:
pd.read_csv("experiments/bkp_metrics.csv")

,Unnamed: 0,code_tag,llm_used,prompt_version,query_id,query_run,sql_query,tab_schema,label,query_gen_time,...,query_results,query_error,sql_time,sql_date,r_row,p_row,r_col,p_col,N_perfect_row,N_perfect_col
0,0,0.16.0,NaN,NaN,8,0,\r\nSELECT\r\n *\r\nFROM\r\n object\r\nI...,NaN,NaN,NaN,...,NaN,NaN,4.471375,2025-05-07T01:10:47,1.0,1.0,1.0,1.0,1.0,1.0
1,1,0.16.0,gpt-4o,v1,8,1,\nSELECT * \nFROM probability \nINNER JOIN obj...,['\nCREATE TABLE object ( /* this is the most ...,medium,13.512856,...,NaN,NaN,4.593240,2025-05-07T01:10:52,0.0,0.0,1.0,1.0,0.0,1.0
2,2,0.16.0,gpt-4o,v1,8,2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0.16.0,gpt-4o,v1,8,3,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.16.0,gpt-4o,v1,8,4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,567,0.16.0,gpt-4o,v1,39,6,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
568,568,0.16.0,gpt-4o,v1,39,7,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
569,569,0.16.0,gpt-4o,v1,39,8,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
570,570,0.16.0,gpt-4o,v1,39,9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
